In [ ]:
import os

import os
from google.colab import drive
drive.mount('/content/drive')
!pip install keras4torch
!pip install einops
import os
os.chdir('/content/drive/MyDrive/ECML2/Stab/5805_s')
#pass

import keras4torch
from   keras4torch.callbacks  import ModelCheckpoint,LRScheduler
import torch
import torch.nn    as nn
import torch.optim as optim
import  torch.nn.functional as     F
import numpy       as np
import pandas      as pd
from copy import deepcopy
import  matplotlib.pyplot   as     plt
from    sklearn.preprocessing import StandardScaler, QuantileTransformer
from    datetime import datetime
import  gc
import STab
from STab import MyClassLoss,CatMap,Num_Cat
from   STab import mainmodel, LWTA, Gsoftmax
MainModel=mainmodel.MainModel

Mounted at /content/drive


In [ ]:

import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Volvo')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
Train=pd.read_csv('train_gen1.csv')
Variants=pd.read_csv('variants.csv')
Out=pd.read_csv('public_X_test.csv')
Variants.set_index('ChassisId_encoded',inplace=True)

In [ ]:
F= []
for c  in Out.columns:
  if c[:2]=='af' or c[:2]=='f_':
    F.append(c)

In [ ]:
os.chdir('/content/drive/MyDrive/VolvoFinal')


In [ ]:
Out.loc[Out.gen=='gen1',F] = (Out.loc[Out.gen=='gen1',F]- Train[F].quantile(0.005))
Out.loc[Out.gen=='gen2',F] = ( Out.loc[Out.gen=='gen2',F]- Out.loc[Out.gen=='gen2',F].quantile(0.005) )
Train[F] = (Train[F]- Train[F].quantile(0.005))


In [ ]:
Train['Tss']  = Train.Timesteps-Train.ChassisId_encoded.map(  Train.groupby('ChassisId_encoded').first().Timesteps)
Out['Tss']    = Out.Timesteps-Out.ChassisId_encoded.map(  Out.groupby('ChassisId_encoded').first().Timesteps)


<ipython-input-9-b045eacdaa2e>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Train['Tss']  = Train.Timesteps-Train.ChassisId_encoded.map(  Train.groupby('ChassisId_encoded').first().Timesteps)


In [ ]:
def BuildXY(Train,Variants,train=True):
  #droplist = ['Timesteps'	,'Tss','ChassisId_encoded',	'gen'	]
  droplist = ['Tss','ChassisId_encoded',	'gen'	]
  if train:
    droplist=droplist+['risk_level']
  TrainV=Variants.loc[Train.ChassisId_encoded]
  TrainV.index=Train.index
  X=Train.drop(droplist,axis=1)#.drop('Timesteps',axis=1)
  A = Train.drop( ['Tss',	'gen'	,'Timesteps'],axis=1)
  if train:

    A = Train.drop( ['Tss','risk_level',	'gen'	,'Timesteps'],axis=1)
  A = A.diff()
  A.loc[A.ChassisId_encoded!=0]=0
  A['ChassisId_encoded']=Train.ChassisId_encoded

  Amean  =  A.groupby('ChassisId_encoded').mean()
  Amean=Amean.loc[Train.ChassisId_encoded]
  Amean.index=Train.index


  #X=pd.concat([X,A,Amean,TrainNorm,TrainV*np.array( [0,0]+[1,1,1,1,1]+[0,0,0,0,0])],axis=1)
 # X=pd.concat([X,TrainV*np.array( [0,0]+[1,1,1,1,1]+[0,0,0,0,0])],axis=1)
  X=pd.concat([X,TrainV*np.array( [0,0]+[0,0,0,0,0]+[0,0,0,0,0])],axis=1)
  #X=pd.concat([X,TrainV*np.array( [0,0]+[1,1,1,1,1]+[0,0,0,0,0])],axis=1)

  X.columns=list(range(len(X.columns)))

  Y=None
  if train:
    L=Train.groupby('ChassisId_encoded').last()
    Train['Y']=60
    Train['sieries_size']= Train['ChassisId_encoded'].map(L.Timesteps)
    Train['sieries_end']= Train['ChassisId_encoded'].map(L.risk_level)
    Train.loc[Train.sieries_end=='High','Y']=1 + Train.loc[Train.sieries_end=='High','sieries_size'] - Train.loc[Train.sieries_end=='High','Timesteps']
    Train.loc[Train.sieries_end=='Low','Y']=Train.loc[Train.sieries_end=='Low','Y'] - Train.loc[Train.sieries_end=='Low','Timesteps']*1
    Y=Train.Y


  return X,Y


In [ ]:
def sample_pairs(data, N):
    # Check if DataFrame and columns exist
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")
    if 'ChassisId_encoded' not in data.columns or 'Timesteps' not in data.columns:
        raise ValueError("DataFrame must contain 'ChassisId_encoded' and 'Timesteps' columns")

    # Store indices and labels
    indices = []
    labels = []

    # Group data by 'ChassisId_encoded'
    grouped = data.groupby('ChassisId_encoded')

    # Create a list to hold possible pairs from each group
    possible_pairs = []
    for _, group in grouped:
        if len(group) >= 2:
            group_indices = group.index.tolist()
            # Generate all possible pairs in the group
            for i in range(len(group_indices)):
                for j in range(i + 1, len(group_indices)):
                    possible_pairs.append((group_indices[i], group_indices[j]))

    # Shuffle the list of possible pairs to randomize selection
    np.random.shuffle(possible_pairs)

    # Select up to N pairs from the shuffled list
    for idx1, idx2 in possible_pairs[:N]:
        # Calculate the label
        label = data.at[idx2, 'Timesteps'] - data.at[idx1, 'Timesteps']
        indices.append([idx1, idx2])
        labels.append(label)
        if len(labels) >= N:
            break

    # Convert results to the requested format
    X = np.array(indices)
    Y = np.array(labels)

    return X, Y

In [ ]:
import random

def myP(y):
  return np.where(y>30, 1.28, 10.00 )

def split_list(L, n, p):
    # Validate that the sum of percentages is approximately 1
    if not (0.99 <= sum(p) <= 1.01):
        raise ValueError("Sum of percentages must be approximately 1")

    # Shuffle the list to randomize the order of elements
    random.shuffle(L)

    # Calculate the sizes of each sublist
    total_length = len(L)
    sizes = [int(total_length * percentage) for percentage in p]

    # Adjust the last element size to match the total length
    sizes[-1] = total_length - sum(sizes[:-1])

    # Split the list into sublists
    result = []
    current_index = 0
    for size in sizes:
        result.append(L[current_index:current_index + size])
        current_index += size

    return result


In [ ]:
X,Y    = BuildXY(Train,Variants,train=True)
Xout,_ = BuildXY(Out,Variants,train=False)

<ipython-input-11-2b85fb24b7ae>:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Train['Y']=60
<ipython-input-11-2b85fb24b7ae>:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Train['sieries_size']= Train['ChassisId_encoded'].map(L.Timesteps)
<ipython-input-11-2b85fb24b7ae>:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `n

In [ ]:

def datasplit(X_,Train_,size=500000,PP=[0.8,0.2]):
  X=deepcopy(X_)
  Train=deepcopy(Train_)
  train_l,test_l=split_list(Train.ChassisId_encoded.unique(),2,PP)
  Itrain=X[Train.ChassisId_encoded.isin(train_l)].index
  Itest=X[Train.ChassisId_encoded.isin(test_l)].index
  X_test,y_test=X.loc[Itest].reset_index().drop('index',axis=1),Y.loc[Itest].reset_index().drop('index',axis=1)
  X_train,y_train=X.loc[Itrain].reset_index().drop('index',axis=1),Y.loc[Itrain].reset_index().drop('index',axis=1)
  prob=myP(y_train)[:,0]


  I=X_train.sample(size, weights=prob,replace=True).index
  X_train = X_train.loc[I]
  y_train = y_train.loc[I]
  y_train=y_train.Y
  return  X_train ,y_train ,X_test,y_test,Itest


In [ ]:
def f1_score(y_true, y_pred, threshold=0.5):
    y_pred = (y_pred > threshold).float()
    true_positives = (y_true * y_pred).sum().to(torch.float32)
    precision = true_positives / (y_pred.sum() + 1e-8)
    recall = true_positives / (y_true.sum() + 1e-8)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    return f1

In [ ]:
import  torch.nn.functional as     F
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


def BuildModel():
    Or_model = MainModel(
        categories        = (),
        num_continuous    = 317,
        dim               = 64  ,
        dim_out           = 2,
        depth             = 2,
        heads             = 8,
        attn_dropout      = 0.25 ,
        ff_dropout        = 0.25,
        U                 = 2,
        cases             = 16

    )
    no_model = Num_Cat(Or_model,num_number=317,classes=2)
    model    = keras4torch.Model(no_model,).build([317])

    return Or_model,no_model,model


def TrainModel(checkpoint,X_train,y_train,X_test,y_test):



  Or_model,no_model,model =BuildModel()
  #Warm UP
  #Warm UP
  optimizer=torch.optim.AdamW(model.parameters(),lr=0.00001,weight_decay=0.0001,)
  sch=torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.001, total_iters=10,  verbose=False)
  model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy])
  callbacks=[LRScheduler(sch)]
  no_model.reset_Sample_size(4)

  model.fit(X_train.fillna(0).values, y_train[0].values,
                        epochs=2, batch_size=512,
                        validation_data=(X_test.fillna(0).values,y_test[0].values),
                        verbose=1,validation_batch_size=64,
                        callbacks=callbacks)
  no_model.reset_Sample_size(8)

  #Main Train
  optimizer=torch.optim.AdamW(model.parameters(),lr=0.001,weight_decay=0.0001,)
  model.compile(optimizer=optimizer, loss=MyClassLoss(0.01,1), metrics=['accuracy', F.cross_entropy])
  scheduler =LRScheduler( torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5,min_lr=0.00001))
  callbacks=[ModelCheckpoint(checkpoint,monitor='val_acc',mode='max'),scheduler]
  model.fit(X_train.fillna(0).values, y_train[0].values,
          epochs=20, batch_size=256,
          validation_data=(X_test.fillna(0).values,y_test[0].values),
          verbose=1,validation_batch_size=128,
          callbacks=callbacks)



  optimizer.zero_grad()  # Zero gradients at the start of your 'batch'
  del no_model
  del Or_model
  del model
  torch.cuda.empty_cache()  # This wi
  gc.collect()

 # model.load_weights(checkpoint)
  return None

In [ ]:
for i in range(5):
  X_train ,y_train ,X_test,y_test,Itest= datasplit(X,Train,size=128000,PP=[0.75,0.25])
  X_train=X_train[(y_train<19) | (y_train>37.5)  ]
  y_train=y_train[(y_train<19) | (y_train>37.5)  ]
  X_test=X_test[(y_test.Y<19) | (y_test.Y>37.5)  ]
  y_test=y_test[(y_test.Y<19) | (y_test.Y>37.5)  ]

  y_test=pd.DataFrame(np.where(y_test>20,0,1).astype(int))
  y_train=pd.DataFrame(np.where(y_train>20,0,1).astype(int))
  TrainModel('LongModel{}'.format(i),X_train,y_train,X_test,y_test)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 100263 samples, validate on 29394 samples:
Epoch 1/2
196/196 - 119s - loss: 0.6874 - acc: 0.5610 - cross_entropy: 0.6935 - val_loss: 0.6674 - val_acc: 0.5856 - val_cross_entropy: 0.6733 - lr: 1e-08
Epoch 2/2
196/196 - 113s - loss: 0.6647 - acc: 0.5906 - cross_entropy: 0.6705 - val_loss: 0.6516 - val_acc: 0.6194 - val_cross_entropy: 0.6573 - lr: 1e-06
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 100263 samples, validate on 29394 samples:
Epoch 1/20
392/392 - 176s - loss: 0.1009 - acc: 0.9635 - cross_entropy: 0.1011 - val_loss: 0.0354 - val_acc: 0.9933 - val_cross_entropy: 0.0348 - lr: 1e-03
Epoch 2/20
392/392 - 176s - loss: 0.0547 - acc: 0.9819 - cross_entropy: 0.0544 - val_loss: 0.0332 - va

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 100046 samples, validate on 28916 samples:
Epoch 1/2
196/196 - 112s - loss: 0.7470 - acc: 0.4869 - cross_entropy: 0.7537 - val_loss: 0.7269 - val_acc: 0.4364 - val_cross_entropy: 0.7334 - lr: 1e-08
Epoch 2/2
196/196 - 114s - loss: 0.7207 - acc: 0.5189 - cross_entropy: 0.7272 - val_loss: 0.6653 - val_acc: 0.5868 - val_cross_entropy: 0.6712 - lr: 1e-06
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 100046 samples, validate on 28916 samples:
Epoch 1/20
391/391 - 177s - loss: 0.0916 - acc: 0.9650 - cross_entropy: 0.0916 - val_loss: 0.0504 - val_acc: 0.9895 - val_cross_entropy: 0.0500 - lr: 1e-03
Epoch 2/20
391/391 - 176s - loss: 0.0495 - acc: 0.9805 - cross_entropy: 0.0491 - val_loss: 0.0452 - va

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 99685 samples, validate on 29468 samples:
Epoch 1/2
195/195 - 113s - loss: 0.6985 - acc: 0.5530 - cross_entropy: 0.7047 - val_loss: 0.7025 - val_acc: 0.5010 - val_cross_entropy: 0.7088 - lr: 1e-08
Epoch 2/2
195/195 - 114s - loss: 0.6756 - acc: 0.5818 - cross_entropy: 0.6816 - val_loss: 0.6403 - val_acc: 0.6422 - val_cross_entropy: 0.6459 - lr: 1e-06
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 99685 samples, validate on 29468 samples:
Epoch 1/20
390/390 - 177s - loss: 0.0900 - acc: 0.9664 - cross_entropy: 0.0900 - val_loss: 0.0736 - val_acc: 0.9764 - val_cross_entropy: 0.0735 - lr: 1e-03
Epoch 2/20
390/390 - 177s - loss: 0.0494 - acc: 0.9829 - cross_entropy: 0.0490 - val_loss: 0.0466 - val_

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 99588 samples, validate on 29352 samples:
Epoch 1/2
195/195 - 114s - loss: 0.7666 - acc: 0.5130 - cross_entropy: 0.7735 - val_loss: 0.9754 - val_acc: 0.1443 - val_cross_entropy: 0.9844 - lr: 1e-08
Epoch 2/2
195/195 - 114s - loss: 0.7223 - acc: 0.5354 - cross_entropy: 0.7287 - val_loss: 0.7783 - val_acc: 0.3289 - val_cross_entropy: 0.7853 - lr: 1e-06
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 99588 samples, validate on 29352 samples:
Epoch 1/20
390/390 - 177s - loss: 0.0996 - acc: 0.9601 - cross_entropy: 0.0997 - val_loss: 0.0543 - val_acc: 0.9830 - val_cross_entropy: 0.0539 - lr: 1e-03
Epoch 2/20
390/390 - 175s - loss: 0.0503 - acc: 0.9806 - cross_entropy: 0.0500 - val_loss: 0.0443 - val_

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 99683 samples, validate on 29094 samples:
Epoch 1/2
195/195 - 112s - loss: 0.7820 - acc: 0.4832 - cross_entropy: 0.7891 - val_loss: 0.5428 - val_acc: 0.8414 - val_cross_entropy: 0.5474 - lr: 1e-08
Epoch 2/2
195/195 - 112s - loss: 0.7469 - acc: 0.4973 - cross_entropy: 0.7536 - val_loss: 0.5996 - val_acc: 0.7482 - val_cross_entropy: 0.6048 - lr: 1e-06
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
Train on 99683 samples, validate on 29094 samples:
Epoch 1/20
390/390 - 175s - loss: 0.1002 - acc: 0.9640 - cross_entropy: 0.1003 - val_loss: 0.0383 - val_acc: 0.9913 - val_cross_entropy: 0.0378 - lr: 1e-03
Epoch 2/20
390/390 - 175s - loss: 0.0538 - acc: 0.9821 - cross_entropy: 0.0534 - val_loss: 0.0402 - val_

In [ ]:
Out=pd.DataFrame()
Or_model,no_model,model= BuildModel()
no_model.reset_Sample_size(1)
for j in range(20):
  for i in range(5):
    model.load_weights('LongModel{}'.format(i))
    Out[i+j*5]=torch.softmax(torch.tensor(model.predict(Xout.fillna(0).values,batch_size=256)),dim=-1)[:,1]


[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto convert float64 to float32, this could lead to extra memory usage.
[Warning] Auto c

In [ ]:
Out.to_csv('OutGlobal.csv')